In [129]:
#!pip install category-encoders
import os
import warnings
from pathlib import Path

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
from IPython.display import display
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
#from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

'''
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
'''
# Mute warnings
warnings.filterwarnings('ignore')

In [130]:
data_dir = Path("dfC.csv")
df = pd.read_csv(data_dir)
df

,date,city,houseType,status,numberRooms,numberBathrooms,numberKitchens,numberBalconies,elevator,carParking,heating,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,price
0,2020-01-29,RB,apartment,A,1,1,1,0,1,1,0,140.0,0,1,0,1300
1,2020-01-29,RB,apartment,A,1,1,1,1,1,1,0,80.0,0,0,1,550
2,2020-01-29,RB,apartment,A,3,3,1,2,1,1,1,190.0,0,0,1,1000
3,2020-01-29,RB,apartment,A,3,3,1,1,1,1,1,190.0,0,0,1,950
4,2020-01-29,RB,apartment,A,3,3,1,3,1,1,0,160.0,0,0,1,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8128,2021-03-03,RB,apartment,A,2,2,1,1,1,1,0,120.0,0,0,1,600
8129,2021-03-03,RB,apartment,B,3,2,1,1,1,1,0,140.0,0,0,1,500
8130,2021-03-03,H,apartment,B,3,2,1,2,0,0,0,150.0,0,1,0,1600
8131,2021-03-03,N,apartment,A,1,1,1,1,1,1,1,70.0,0,1,0,1000


In [131]:
def impute(df):
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0)
    for name in df.select_dtypes("category"):
        df[name] = df[name].fillna("None")
    return df



impute(df)



,date,city,houseType,status,numberRooms,numberBathrooms,numberKitchens,numberBalconies,elevator,carParking,heating,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,price
0,2020-01-29,RB,apartment,A,1,1,1,0,1,1,0,140.0,0,1,0,1300
1,2020-01-29,RB,apartment,A,1,1,1,1,1,1,0,80.0,0,0,1,550
2,2020-01-29,RB,apartment,A,3,3,1,2,1,1,1,190.0,0,0,1,1000
3,2020-01-29,RB,apartment,A,3,3,1,1,1,1,1,190.0,0,0,1,950
4,2020-01-29,RB,apartment,A,3,3,1,3,1,1,0,160.0,0,0,1,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8128,2021-03-03,RB,apartment,A,2,2,1,1,1,1,0,120.0,0,0,1,600
8129,2021-03-03,RB,apartment,B,3,2,1,1,1,1,0,140.0,0,0,1,500
8130,2021-03-03,H,apartment,B,3,2,1,2,0,0,0,150.0,0,1,0,1600
8131,2021-03-03,N,apartment,A,1,1,1,1,1,1,1,70.0,0,1,0,1000


In [132]:
# Peek at the values
#display(df)

# Display information about dtypes and missing values

display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8133 entries, 0 to 8132
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             8133 non-null   object 
 1   city             8133 non-null   object 
 2   houseType        8133 non-null   object 
 3   status           8133 non-null   object 
 4   numberRooms      8133 non-null   int64  
 5   numberBathrooms  8133 non-null   int64  
 6   numberKitchens   8133 non-null   int64  
 7   numberBalconies  8133 non-null   int64  
 8   elevator         8133 non-null   int64  
 9   carParking       8133 non-null   int64  
 10  heating          8133 non-null   int64  
 11  houseSpace       8133 non-null   float64
 12  PriceType_JOD    8133 non-null   int64  
 13  PriceType_NIS    8133 non-null   int64  
 14  PriceType_USD    8133 non-null   int64  
 15  price            8133 non-null   int64  
dtypes: float64(1), int64(11), object(4)
memory usage: 1016.8+ KB

None

In [133]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [134]:
X = df.copy()
y = X.pop("price")

mi_scores = make_mi_scores(X, y)
mi_scores

PriceType_USD      0.373450
PriceType_NIS      0.289018
city               0.228781
houseSpace         0.191795
PriceType_JOD      0.174511
numberRooms        0.111146
numberBathrooms    0.105612
status             0.096369
elevator           0.060125
heating            0.054981
carParking         0.046087
numberBalconies    0.039944
date               0.036985
numberKitchens     0.004049
houseType          0.003666
Name: MI Scores, dtype: float64

In [135]:
def drop_uninformative(df, mi_scores):
    return df.loc[:, mi_scores > 0.006]

In [136]:
X = df.copy()
y = X.pop("price")
X = drop_uninformative(X, mi_scores)

mi_scores = make_mi_scores(X, y)
mi_scores

PriceType_USD      0.373450
PriceType_NIS      0.289018
city               0.228781
houseSpace         0.191795
PriceType_JOD      0.174511
numberRooms        0.111146
numberBathrooms    0.105612
status             0.096369
elevator           0.060125
heating            0.054981
carParking         0.046087
numberBalconies    0.039944
date               0.036985
Name: MI Scores, dtype: float64

In [137]:
def create_features(df):
    X = df.copy()
    y = X.pop("price")
    X = X.join(create_features_1(X))
    X = X.join(create_features_2(X))
    X = X.join(create_features_3(X))
    # ...
    return X


def label_encode(df):
    X = df.copy()
    for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes
    return X





In [138]:
def mathematical_transforms(df):
    X = pd.DataFrame()  # dataframe to hold new features
    X["featurePlus"] = df.numberBathrooms / df.numberRooms
    #X["featurePlus2"] = (df.numberBalconies + df.numberBathrooms) / df.numberRooms
    # This feature ended up not helping performance
    
    return X


def interactions(df):
    X = pd.get_dummies(df.status, prefix="Bldg")
    X = X.mul(df.numberBalconies, axis=0)

    return X


def group_transforms(df):
    X = pd.DataFrame()
    X["MedstatusHouseSpace"] = df.groupby("status")["houseSpace"].transform("median")
    return X

In [139]:
def corrplot(df, method="pearson", annot=True, **kwargs):
    sns.clustermap(
        df.corr(method),
        vmin=-1.0,
        vmax=1.0,
        cmap="icefire",
        method="complete",
        annot=annot,
        **kwargs,
    )

In [140]:
class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=5)

    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

In [141]:
def create_features(df, df_test=None):
    X = df.copy()
    y = X.pop("price")
    mi_scores = make_mi_scores(X, y)

    if df_test is not None:
        X_test = df_test.copy()
        X_test.pop("SalePrice")
        X = pd.concat([X, X_test])

    #Mutual Information
    X = drop_uninformative(X, mi_scores)

    #Transformations
    X = X.join(mathematical_transforms(X))
    X = X.join(interactions(X))
 
    X = X.join(group_transforms(X))

    # Reform splits
    if df_test is not None:
        X_test = X.loc[df_test.index, :]
        X.drop(df_test.index, inplace=True)

    # Encoder
    encoder = CrossFoldEncoder(MEstimateEncoder, m=1)
    X = X.join(encoder.fit_transform(X, y, cols=["city"]))
    if df_test is not None:
        X_test = X_test.join(encoder.transform(X_test))

    if df_test is not None:
        return X, X_test
    else:
        return X


#df_train, df_test = load_data()
X = create_features(df)
y = df.loc[:, "price"]


In [239]:
result = pd.concat([X, y], axis=1)

In [240]:
result

,date,city,status,numberRooms,numberBathrooms,numberBalconies,elevator,carParking,heating,houseSpace,...,PriceType_USD,featurePlus,Bldg_A,Bldg_B,Bldg_C,Bldg_D,Bldg_E,MedstatusHouseSpace,city_encoded,price
0,2020-01-29,RB,A,1,1,0,1,1,0,140.0,...,0,1.000000,0,0,0,0,0,140.0,2179.295502,1300
1,2020-01-29,RB,A,1,1,1,1,1,0,80.0,...,1,1.000000,1,0,0,0,0,140.0,2179.295502,550
2,2020-01-29,RB,A,3,3,2,1,1,1,190.0,...,1,1.000000,2,0,0,0,0,140.0,2179.295502,1000
3,2020-01-29,RB,A,3,3,1,1,1,1,190.0,...,1,1.000000,1,0,0,0,0,140.0,2179.295502,950
4,2020-01-29,RB,A,3,3,3,1,1,0,160.0,...,1,1.000000,3,0,0,0,0,140.0,2179.295502,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8128,2021-03-03,RB,A,2,2,1,1,1,0,120.0,...,1,1.000000,1,0,0,0,0,140.0,1789.162426,600
8129,2021-03-03,RB,B,3,2,1,1,1,0,140.0,...,1,0.666667,0,1,0,0,0,140.0,1789.162426,500
8130,2021-03-03,H,B,3,2,2,0,0,0,150.0,...,0,0.666667,0,2,0,0,0,140.0,4696.623612,1600
8131,2021-03-03,N,A,1,1,1,1,1,1,70.0,...,0,1.000000,1,0,0,0,0,140.0,3882.401112,1000


In [241]:
mi_scores = make_mi_scores(X, y)
mi_scores

PriceType_USD          0.366945
PriceType_NIS          0.281812
city_encoded           0.258938
city                   0.233117
houseSpace             0.194606
PriceType_JOD          0.176285
numberRooms            0.117014
numberBathrooms        0.101102
status                 0.082846
featurePlus            0.075259
elevator               0.063515
Bldg_A                 0.060133
carParking             0.051842
heating                0.049441
Bldg_B                 0.047798
date                   0.041258
numberBalconies        0.040540
Bldg_C                 0.028869
MedstatusHouseSpace    0.010620
Bldg_D                 0.005140
Bldg_E                 0.004738
Name: MI Scores, dtype: float64

In [242]:
#result.to_csv (r'dfFE.csv', index = False, header=True)

result.describe()

,numberRooms,numberBathrooms,numberBalconies,elevator,carParking,heating,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,featurePlus,Bldg_A,Bldg_B,Bldg_C,Bldg_D,Bldg_E,MedstatusHouseSpace,city_encoded,price
count,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000,8133.000000
mean,2.491209,1.860445,1.022747,0.599656,0.654002,0.221812,180.747572,0.100209,0.410427,0.489241,0.787071,0.387065,0.557728,0.070454,0.004672,0.002828,140.023362,2697.913941,2720.654986
std,0.864293,0.738796,0.739643,0.489998,0.475722,0.415491,2818.908055,0.300297,0.491941,0.499915,0.283813,0.644466,0.761221,0.345484,0.087192,0.074335,0.482803,1588.000928,22107.738119
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-140.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,140.000000,975.620530,1.000000
25%,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,120.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,140.000000,2036.092525,500.000000
50%,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,140.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,140.000000,2179.295502,800.000000
75%,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,150.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,140.000000,2333.598953,1400.000000
max,6.000000,4.000000,5.000000,1.000000,1.000000,1.000000,190000.000000,1.000000,1.000000,1.000000,3.000000,4.000000,5.000000,5.000000,2.000000,3.000000,150.000000,24291.482826,1000000.000000


In [243]:
#drop extreme data
index_names = result[ result['city'] == 'TB' ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True) 

In [244]:
#drop extreme data
index_names = result[ result['city'] == 'S' ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True) 

In [245]:
#drop extreme data
index_names = result[ result['price'] < 200 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True) 

In [246]:
#drop extreme data
index_names = result[ result['price'] > 4000 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True) 

In [247]:
#drop extreme data
index_names = result[ result['city_encoded'] < 1600 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True) 

In [248]:
#drop extreme data
index_names = result[ result['city_encoded'] > 10000 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True) 

In [249]:
#drop extreme data
index_names = result[ result['Bldg_C'] > 2 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True) 

In [250]:
#drop extreme data
index_names = result[ result['Bldg_B'] > 3 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True) 

In [251]:
#drop extreme data
index_names = result[ result['Bldg_A'] > 3 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True)

In [252]:
#drop extreme data
index_names = result[ result['featurePlus'] > 2 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True)

In [253]:
#drop extreme data
index_names = result[ result['featurePlus'] < 0.2 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True)

In [254]:
#drop extreme data
index_names = result[ result['houseSpace'] > 250 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True)

In [255]:
#drop extreme data
index_names = result[ result['houseSpace'] < 20 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True)

In [256]:
#drop extreme data
index_names = result[ result['numberBathrooms'] > 3 ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True)

In [268]:
result

,city,status,numberRooms,numberBathrooms,numberBalconies,elevator,carParking,heating,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,featurePlus,Bldg_A,Bldg_B,Bldg_C,MedstatusHouseSpace,city_encoded,price
0,1,1,1,1,0,1,1,0,140.0,0,1,0,1.000000,0,0,0,140.0,2179.295502,1300
1,1,1,1,1,1,1,1,0,80.0,0,0,1,1.000000,1,0,0,140.0,2179.295502,550
2,1,1,3,3,2,1,1,1,190.0,0,0,1,1.000000,2,0,0,140.0,2179.295502,1000
3,1,1,3,3,1,1,1,1,190.0,0,0,1,1.000000,1,0,0,140.0,2179.295502,950
4,1,1,3,3,3,1,1,0,160.0,0,0,1,1.000000,3,0,0,140.0,2179.295502,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8128,1,1,2,2,1,1,1,0,120.0,0,0,1,1.000000,1,0,0,140.0,1789.162426,600
8129,1,2,3,2,1,1,1,0,140.0,0,0,1,0.666667,0,1,0,140.0,1789.162426,500
8130,5,2,3,2,2,0,0,0,150.0,0,1,0,0.666667,0,2,0,140.0,4696.623612,1600
8131,4,1,1,1,1,1,1,1,70.0,0,1,0,1.000000,1,0,0,140.0,3882.401112,1000


In [258]:
result['city'] = result['city'].replace(['RB'],'1')
result['city'] = result['city'].replace(['T'],'2')
result['city'] = result['city'].replace(['B'],'3')
result['city'] = result['city'].replace(['N'],'4')
result['city'] = result['city'].replace(['H'],'5')
result['city'] = result['city'].replace(['J'],'6')
result['city'] = result['city'].replace(['JN'],'7')
result['city'] = result['city'].replace(['JR'],'8')
result['city'] = result['city'].replace(['Q'],'9')

In [259]:
result['status'] = result['status'].replace(['A'],'1')
result['status'] = result['status'].replace(['B'],'2')
result['status'] = result['status'].replace(['C'],'3')
result['status'] = result['status'].replace(['D'],'4')
result['status'] = result['status'].replace(['E'],'5')

In [260]:
result = result.drop(['Bldg_D', 'Bldg_E'], axis=1)
result = result.drop(['date'], axis=1)
result

,city,status,numberRooms,numberBathrooms,numberBalconies,elevator,carParking,heating,houseSpace,PriceType_JOD,PriceType_NIS,PriceType_USD,featurePlus,Bldg_A,Bldg_B,Bldg_C,MedstatusHouseSpace,city_encoded,price
0,1,1,1,1,0,1,1,0,140.0,0,1,0,1.000000,0,0,0,140.0,2179.295502,1300
1,1,1,1,1,1,1,1,0,80.0,0,0,1,1.000000,1,0,0,140.0,2179.295502,550
2,1,1,3,3,2,1,1,1,190.0,0,0,1,1.000000,2,0,0,140.0,2179.295502,1000
3,1,1,3,3,1,1,1,1,190.0,0,0,1,1.000000,1,0,0,140.0,2179.295502,950
4,1,1,3,3,3,1,1,0,160.0,0,0,1,1.000000,3,0,0,140.0,2179.295502,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8128,1,1,2,2,1,1,1,0,120.0,0,0,1,1.000000,1,0,0,140.0,1789.162426,600
8129,1,2,3,2,1,1,1,0,140.0,0,0,1,0.666667,0,1,0,140.0,1789.162426,500
8130,5,2,3,2,2,0,0,0,150.0,0,1,0,0.666667,0,2,0,140.0,4696.623612,1600
8131,4,1,1,1,1,1,1,1,70.0,0,1,0,1.000000,1,0,0,140.0,3882.401112,1000


In [261]:
#sns.relplot(x="city", y="price", data=result);
#drop extreme data
index_names = result[ result['city'] == '8' ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True)


In [262]:
#sns.relplot(x="city", y="price", data=result);
#drop extreme data
index_names = result[ result['status'] == '5' ].index 
index_names
# drop these row indexes 
# from dataFrame 
result.drop(index_names, inplace = True)


In [267]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7566 entries, 0 to 8132
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   city                 7566 non-null   int32  
 1   status               7566 non-null   int32  
 2   numberRooms          7566 non-null   int64  
 3   numberBathrooms      7566 non-null   int64  
 4   numberBalconies      7566 non-null   int64  
 5   elevator             7566 non-null   int64  
 6   carParking           7566 non-null   int64  
 7   heating              7566 non-null   int64  
 8   houseSpace           7566 non-null   float64
 9   PriceType_JOD        7566 non-null   int64  
 10  PriceType_NIS        7566 non-null   int64  
 11  PriceType_USD        7566 non-null   int64  
 12  featurePlus          7566 non-null   float64
 13  Bldg_A               7566 non-null   int64  
 14  Bldg_B               7566 non-null   int64  
 15  Bldg_C               7566 non-null   i

In [266]:
result["status"] = result["status"].astype(str).astype(int)
result["city"] = result["city"].astype(str).astype(int)

In [269]:
result.to_csv (r'dfFE11.csv', index = False, header=True)